# Social Engineering Scam Detection with LLMs

Ronish Nair, Fahim Abbasi, and Shahbaz Pervez. 2025. PhishEmailLLM:A Meta Model Approach to Detect Phishing emails by leveraging LLMs and Machine Learning models. In 2025 Australasian Computer Science Week (ACSW 2025), February 10–13, 2025, Brisbane, QLD, Australia. ACM, New York, NY, USA, 11 pages. https://doi.org/10.1145/3727166.3727169

In [ ]:
import os 
os.getcwd()

### Imports

In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
nltk.download("stopwords")

### Load Datasets

In [ ]:
#Enron Public Email Corpus

#Spam Assassin Corpus

#Phishing Pot

### Data Pre-processing 

###  Feature Extraction

### Models

Training/Testing

In [ ]:
# Llama3.1

In [ ]:
# Llama3.2

In [ ]:
# Gemma2

In [ ]:
# Qwen2.5

In [ ]:
# Mistral

In [ ]:
# Granite3 Dense

Evaluation

In [ ]:
# Precision

# Recall

# Accuracy

# F1